In [ ]:
# default_exp regularization.regularization_callback

# RegularizationCallback

> Perform Group Regularization in fastai Callback system

In [ ]:
#all_slow

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
from fastai.vision.all import *
from fastai.callback.all import *
from fasterai.sparse.criteria import *

import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
#hide
path = untar_data(URLs.PETS)
files = get_image_files(path/"images")

def label_func(f): return f[0].isupper()

dls = ImageDataLoaders.from_name_func(path, files, label_func, item_tfms=Resize(64))

In [ ]:
learn = cnn_learner(dls, resnet18, metrics=accuracy)
learn.unfreeze()

learn.fit_one_cycle(3)

epoch,train_loss,valid_loss,accuracy,time
0,0.681536,0.466989,0.835589,00:11
1,0.358927,0.318825,0.865359,00:10
2,0.201207,0.220008,0.923545,00:10


In [ ]:
#export
class RegularizationCallback(Callback):
    def __init__(self, granularity, wd=0.01):
        store_attr()

    def after_loss(self):
        reg = self.get_norm()
        self.learn.loss_grad += reg
        self.learn.loss = self.learn.loss_grad.clone()
        
    def get_norm(self):
        return self.wd*torch.stack([large_final(m, self.granularity).sum() for m in learn.modules() if isinstance(m, nn.Conv2d)]).sum()

The `RegularizationCallback`can be used to perform $l_1$ regularization on any granularity available in the `criteria` class.

In [ ]:
reg_cb = RegularizationCallback('filter')

In [ ]:
learn = cnn_learner(dls, resnet18, metrics=accuracy)
learn.unfreeze()

learn.fit_one_cycle(3, cbs=reg_cb)

epoch,train_loss,valid_loss,accuracy,time
0,1.633497,1.468702,0.812585,00:10
1,1.334702,1.173871,0.907307,00:10
2,1.152696,1.136654,0.933694,00:11
